In this notebook we design a multi-modal model using GraphSAGE.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/Colab\ Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras
from keras.layers import Dense
from keras.models import Sequential
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torchtext
import torch
from torch.nn import Linear
import itertools
import nltk
import spacy
import random
from tqdm import tqdm
import time
from IPython.display import Javascript
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve, classification_report

In [ ]:
import numpy as np
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import math
from collections import Counter
import random
import csv
import os
import sys

In [ ]:
random.seed(10)

In [ ]:
df = pd.read_csv('./datasets/gentrification_labels.csv')

In [ ]:
df.tail(10)

,GEOID,label,the_geom,STATEFP10,COUNTYFP10,TRACTCE10,NAME10,NAMELSAD10,INTPTLAT,INTPTLONG,long+,long-,lat+,lat-
791,17031843200,1,MULTIPOLYGON (((-87.65045099997508 41.85258499...,17,31,843200,8432.0,Census Tract 8432,41.848715,-87.664217,-87.660597,-87.667837,41.851420,41.846010
792,17031843300,1,MULTIPOLYGON (((-87.68821699998962 41.85933700...,17,31,843300,8433.0,Census Tract 8433,41.863764,-87.692550,-87.688930,-87.696170,41.866469,41.861059
793,17031843400,1,MULTIPOLYGON (((-87.72356499996559 41.85156999...,17,31,843400,8434.0,Census Tract 8434,41.849327,-87.732022,-87.728402,-87.735642,41.852032,41.846622
794,17031843500,1,MULTIPOLYGON (((-87.6927909999821 41.844721999...,17,31,843500,8435.0,Census Tract 8435,41.840983,-87.697566,-87.693946,-87.701186,41.843688,41.838278
795,17031843600,0,MULTIPOLYGON (((-87.61128600000754 41.80216899...,17,31,843600,8436.0,Census Tract 8436,41.806765,-87.608949,-87.605329,-87.612569,41.809470,41.804060
796,17031843700,0,MULTIPOLYGON (((-87.69666899996376 41.94854100...,17,31,843700,8437.0,Census Tract 8437,41.945560,-87.690034,-87.686414,-87.693654,41.948265,41.942855
797,17031843800,0,MULTIPOLYGON (((-87.6451599999826 41.794302000...,17,31,843800,8438.0,Census Tract 8438,41.801657,-87.640476,-87.636856,-87.644096,41.804362,41.798952
798,17031843900,0,MULTIPOLYGON (((-87.55868599996295 41.77379199...,17,31,843900,8439.0,Census Tract 8439,41.778993,-87.576130,-87.572510,-87.579750,41.781698,41.776288
799,17031980000,0,MULTIPOLYGON (((-87.92062799997296 42.00453199...,17,31,980000,9800.0,Census Tract 9800,41.979419,-87.902438,-87.898818,-87.906058,41.982124,41.976714
800,17031980100,0,MULTIPOLYGON (((-87.73789600001243 41.78578500...,17,31,980100,9801.0,Census Tract 9801,41.785981,-87.750900,-87.747280,-87.754520,41.788686,41.783276


In [ ]:
ids = df['TRACTCE10']
labels = df['label']
lats = df['INTPTLAT']
longs = df['INTPTLONG']

In [ ]:
econ_df = pd.read_csv('./datasets/use_feature_chicago.csv')
econ_df.tail(10)

,Unnamed: 0,HEALTH INSURANCE COVERAGE With private health insurance,Median age (years),Rental vacancy rate,GROSS RENT Occupied units paying rent Median (dollars),Own children of the householder 6 to 17 years,YEAR OF ENTRY Foreign born,SELECTED MONTHLY OWNER COSTS (SMOC) Housing units without a mortgage Median (dollars),relation_max_fee,Own children of the householder under 6 years,...,MARITAL STATUS Females 15 years and over Never married,Percent bachelor's degree or higher,MORTGAGE STATUS Owner-occupied units Housing units with a mortgage,White,"Arts, entertainment, and recreation, and accommodation and food services",HOUSING TENURE Renter-occupied,Hispanic or Latino (of any race),"Educational services, and health care and social assistance",GEOID,label
789,789,0.19,24.1,10.8,880.0,616.0,40.0,673.0,1096.0,345.0,...,0.68,0.08,0.44,0.00,0.08,0.88,0.02,0.24,1.703184e+10,0.0
790,790,0.39,30.6,9.6,984.0,361.0,20.0,0.0,1015.0,158.0,...,0.67,0.17,0.65,0.09,0.10,0.76,0.02,0.35,1.703184e+10,0.0
791,791,0.48,31.4,5.9,872.0,295.0,785.0,570.0,1022.0,226.0,...,0.49,0.30,0.57,0.55,0.14,0.71,0.76,0.23,1.703184e+10,1.0
792,792,0.42,30.2,5.4,956.0,194.0,383.0,769.0,1412.0,234.0,...,0.51,0.16,0.71,0.35,0.05,0.75,0.36,0.26,1.703184e+10,1.0
793,793,0.38,35.1,10.7,667.0,197.0,83.0,516.0,1002.0,89.0,...,0.54,0.08,0.33,0.14,0.07,0.60,0.20,0.17,1.703184e+10,1.0
794,794,0.26,29.4,3.4,767.0,115.0,919.0,528.0,791.0,186.0,...,0.69,0.03,0.36,0.27,0.26,0.58,0.24,0.14,1.703184e+10,1.0
795,795,0.44,35.7,8.9,1040.0,510.0,178.0,489.0,1090.0,304.0,...,0.51,0.38,0.79,0.06,0.06,0.86,0.04,0.29,1.703184e+10,0.0
796,796,0.84,34.4,3.4,1264.0,438.0,480.0,836.0,1570.0,256.0,...,0.38,0.53,0.75,0.85,0.08,0.38,0.27,0.15,1.703184e+10,0.0
797,797,0.45,38.3,9.8,1182.0,351.0,220.0,583.0,921.0,53.0,...,0.44,0.07,0.70,0.37,0.10,0.48,0.24,0.10,1.703184e+10,0.0
798,798,0.54,48.4,21.8,963.0,307.0,266.0,824.0,1207.0,109.0,...,0.52,0.39,0.64,0.04,0.05,0.70,0.03,0.42,1.703184e+10,0.0


In [ ]:
object_df = pd.read_csv('./datasets/chicago_object_detection.csv')
object_df

,tract_id,house_num,shape_area,shape_length,target
0,17031100200,3215,6.454590e+06,543088.273624,0
1,17031100300,2932,7.133997e+06,518288.477552,0
2,17031100400,2089,3.885925e+06,312156.844091,0
3,17031100500,3414,6.531967e+06,541545.440824,0
4,17031100600,2938,4.269677e+06,435811.526911,0
...,...,...,...,...,...
778,17031081800,136,3.220261e+06,83670.173764,0
779,17031090100,2399,3.769413e+06,371042.398789,0
780,17031090200,2977,5.526441e+06,497302.590780,0
781,17031090300,848,1.966836e+06,140454.460942,0


In [ ]:
shortest_df = pd.read_csv('./datasets/chicago_shortest_paths.csv')
shortest_df

,id,hospital_1,hospital_2,subway_1,subway_2,school_1,school_2
0,1.703101e+10,5.0,11.0,1,1,1,1
1,1.703101e+10,2.0,10.0,2,2,1,1
2,1.703101e+10,3.0,11.0,1,1,1,1
3,1.703101e+10,5.0,10.0,1,2,1,1
4,1.703101e+10,7.0,8.0,1,2,1,2
...,...,...,...,...,...,...,...
760,1.703184e+10,2.0,5.0,2,3,1,1
761,1.703184e+10,6.0,6.0,4,5,1,1
762,1.703184e+10,6.0,6.0,2,3,1,1
763,1.703184e+10,1.0,5.0,6,8,2,2


In [ ]:
combined_df = pd.merge(econ_df, object_df, left_on='GEOID', right_on='tract_id')
combined_df = pd.merge(combined_df, df, left_on='GEOID', right_on='GEOID')
combined_df = pd.merge(combined_df, shortest_df, left_on='GEOID', right_on='id')
combined_df

,Unnamed: 0,HEALTH INSURANCE COVERAGE With private health insurance,Median age (years),Rental vacancy rate,GROSS RENT Occupied units paying rent Median (dollars),Own children of the householder 6 to 17 years,YEAR OF ENTRY Foreign born,SELECTED MONTHLY OWNER COSTS (SMOC) Housing units without a mortgage Median (dollars),relation_max_fee,Own children of the householder under 6 years,...,long-,lat+,lat-,id,hospital_1,hospital_2,subway_1,subway_2,school_1,school_2
0,0,0.62,33.7,10.3,865.0,495.0,733.0,650.0,1003.0,398.0,...,-87.673452,42.023961,42.018551,1.703101e+10,5.0,11.0,1,1,1,1
1,1,0.43,34.8,5.3,989.0,934.0,2663.0,714.0,1076.0,644.0,...,-87.683768,42.018713,42.013303,1.703101e+10,2.0,10.0,2,2,1,1
2,2,0.43,36.7,7.0,943.0,185.0,963.0,813.0,1003.0,210.0,...,-87.676942,42.018756,42.013346,1.703101e+10,3.0,11.0,1,1,1,1
3,3,0.60,42.1,5.5,948.0,372.0,1566.0,597.0,990.0,395.0,...,-87.670155,42.018646,42.013236,1.703101e+10,5.0,10.0,1,2,1,1
4,4,0.81,23.8,11.5,979.0,348.0,585.0,651.0,990.0,306.0,...,-87.662436,42.009116,42.003706,1.703101e+10,7.0,8.0,1,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,794,0.26,29.4,3.4,767.0,115.0,919.0,528.0,791.0,186.0,...,-87.701186,41.843688,41.838278,1.703184e+10,4.0,5.0,4,4,1,1
752,795,0.44,35.7,8.9,1040.0,510.0,178.0,489.0,1090.0,304.0,...,-87.612569,41.809470,41.804060,1.703184e+10,2.0,5.0,2,3,1,1
753,796,0.84,34.4,3.4,1264.0,438.0,480.0,836.0,1570.0,256.0,...,-87.693654,41.948265,41.942855,1.703184e+10,6.0,6.0,4,5,1,1
754,797,0.45,38.3,9.8,1182.0,351.0,220.0,583.0,921.0,53.0,...,-87.644096,41.804362,41.798952,1.703184e+10,6.0,6.0,2,3,1,1


In [ ]:
combined_df.columns

Index(['Unnamed: 0', 'HEALTH INSURANCE COVERAGE With private health insurance',
       'Median age (years)', 'Rental vacancy rate',
       'GROSS RENT Occupied units paying rent Median (dollars)',
       'Own children of the householder 6 to 17 years',
       'YEAR OF ENTRY Foreign born',
       'SELECTED MONTHLY OWNER COSTS (SMOC) Housing units without a mortgage Median (dollars)',
       'relation_max_fee', 'Own children of the householder under 6 years',
       'Total population', 'relation_min_fee', 'VALUE Median (dollars)',
       'Mean household income (dollars)', 'Total housing units',
       'SELECTED MONTHLY OWNER COSTS (SMOC) Housing units with a mortgage Median (dollars)',
       'Homeowner vacancy rate', 'Mean travel time to work (minutes)',
       'Sex ratio (males per 100 females)', 'ROOM Median rooms',
       'MARITAL STATUS Females 15 years and over Never married',
       'Percent bachelor's degree or higher',
       'MORTGAGE STATUS Owner-occupied units Housing units w

In [ ]:
combined_df.isnull().any()

Unnamed: 0                                                                               False
HEALTH INSURANCE COVERAGE With private health insurance                                  False
Median age (years)                                                                       False
Rental vacancy rate                                                                      False
GROSS RENT Occupied units paying rent Median (dollars)                                   False
Own children of the householder 6 to 17 years                                            False
YEAR OF ENTRY Foreign born                                                               False
SELECTED MONTHLY OWNER COSTS (SMOC) Housing units without a mortgage Median (dollars)    False
relation_max_fee                                                                         False
Own children of the householder under 6 years                                            False
Total population                                  

In [ ]:
ids = combined_df['tract_id']
labels = combined_df['label_x']
lats = combined_df['INTPTLAT']
longs = combined_df['INTPTLONG']
econ_features = combined_df[['HEALTH INSURANCE COVERAGE With private health insurance',
        'Median age (years)', 'Rental vacancy rate',
        'GROSS RENT Occupied units paying rent Median (dollars)',
        'Own children of the householder 6 to 17 years',
        'YEAR OF ENTRY Foreign born',
        'SELECTED MONTHLY OWNER COSTS (SMOC) Housing units without a mortgage Median (dollars)',
        'relation_max_fee', 'Own children of the householder under 6 years',
        'Total population', 'relation_min_fee', 'VALUE Median (dollars)',
        'Mean household income (dollars)', 'Total housing units',
        'SELECTED MONTHLY OWNER COSTS (SMOC) Housing units with a mortgage Median (dollars)',
        'Homeowner vacancy rate', 'Mean travel time to work (minutes)',
        'Sex ratio (males per 100 females)', 'ROOM Median rooms',
        'MARITAL STATUS Females 15 years and over Never married',
        "Percent bachelor's degree or higher",
        'MORTGAGE STATUS Owner-occupied units Housing units with a mortgage',
        'White',
        'Arts, entertainment, and recreation, and accommodation and food services',
        'HOUSING TENURE Renter-occupied', 'Hispanic or Latino (of any race)',
        'Educational services, and health care and social assistance',
        'INTPTLAT', 'INTPTLONG']]
image_features = combined_df[['house_num', 'shape_area']]
path_features = combined_df[['hospital_1', 'hospital_2', 'subway_1', 'subway_2', 'school_1', 'school_2']]

In [ ]:
chi_lat_n = 42.05
chi_lat_s = 41.7
chi_long_w = -87.9
chi_long_e = -87.53

In [ ]:
tracts = pd.DataFrame(list(zip(ids, lats, longs, labels)))
tracts.columns = ['id', 'lat', 'long', 'label']

In [ ]:
tracts.describe()

,id,lat,long,label
count,7.560000e+02,756.000000,756.000000,756.000000
mean,1.703139e+10,41.867148,-87.678968,0.199735
std,2.767405e+05,0.083613,0.057800,0.400066
min,1.703101e+10,41.700855,-87.844849,0.000000
25%,1.703115e+10,41.791105,-87.715911,0.000000
50%,1.703131e+10,41.878460,-87.675091,0.000000
75%,1.703166e+10,41.936104,-87.641480,0.000000
max,1.703184e+10,42.021256,-87.530703,1.000000


In [ ]:
tracts.tail(10)

,id,lat,long,label
746,17031842600,41.816219,-87.651695,0.0
747,17031842800,41.807534,-87.698304,0.0
748,17031843100,41.862602,-87.699453,0.0
749,17031843300,41.863764,-87.692550,1.0
750,17031843400,41.849327,-87.732022,1.0
751,17031843500,41.840983,-87.697566,1.0
752,17031843600,41.806765,-87.608949,0.0
753,17031843700,41.945560,-87.690034,0.0
754,17031843800,41.801657,-87.640476,0.0
755,17031843900,41.778993,-87.576130,0.0


In [ ]:
for index, row in tracts.iterrows():
    if (row['lat'] > chi_lat_n or row['lat'] < chi_lat_s or
            row['long'] > chi_long_e or row['long'] < chi_long_w):
        tracts = tracts.drop([index])

In [ ]:
pip install pykml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 152 kB/s 


In [ ]:
from pykml import parser

In [ ]:
def get_distance(p1, p2):
    dist = math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)
    return dist

In [ ]:
filepath = './datasets/kmlHospitals.kml'
with open(filepath) as fobj:
    folder = parser.parse(fobj).getroot().Document.Folder

hospitals = []
for pm in folder.Placemark:
    coordinates = pm.Point.coordinates
    longitude = float(str(coordinates)[0:11])
    latitude = float(str(coordinates)[12:21])
    if ((latitude > chi_lat_s and latitude < chi_lat_n) and
            (longitude > chi_long_w and longitude < chi_long_e)):
        hospitals.append({"longitude": longitude, "latitude": latitude, "label":0, "closest": -1})

In [ ]:
for i, hospital in enumerate(hospitals):
    closest_distance = 1000000
    for index, row in tracts.iterrows():
        dist = get_distance((row['long'], row['lat']),
                            (hospital["longitude"], hospital["latitude"]))
        if dist < closest_distance:
            closest_distance = dist
            label = row['label']
            closest_i = index
    hospitals[i]["label"] = int(label)
    hospitals[i]["closest"] = int(closest_i)

In [ ]:
filepath = './datasets/CTA_RailStations.kml'
with open(filepath) as fobj:
    folder = parser.parse(fobj).getroot().Document.Folder

cta = []
for pm in folder.Placemark:
    coordinates = str(pm.Point.coordinates).split(',')
    longitude = float(coordinates[0])
    latitude = float(coordinates[1])
    if ((latitude > chi_lat_s and latitude < chi_lat_n) and
            (longitude > chi_long_w and longitude < chi_long_e)):
        cta.append({"longitude": longitude, "latitude": latitude, "label":0, "closest": -1})

In [ ]:
for i, station in enumerate(cta):
    closest_distance = 1000000
    for index, row in tracts.iterrows():
        dist = get_distance((row['long'], row['lat']),
                            (station["longitude"], station["latitude"]))
        if dist < closest_distance:
            closest_distance = dist
            label = row['label']
            closest_i = index
    cta[i]["label"] = int(label)
    cta[i]["closest"] = int(closest_i)

In [ ]:
filepath = './datasets/kmlPrivateSchools.kml'
with open(filepath) as fobj:
    folder = parser.parse(fobj).getroot().Document.Folder

private_schools = []
for pm in folder.Placemark:
    coordinates = pm.Point.coordinates
    longitude = float(str(coordinates)[0:11])
    latitude = float(str(coordinates)[12:21])
    if ((latitude > chi_lat_s and latitude < chi_lat_n) and
            (longitude > chi_long_w and longitude < chi_long_e)):
        private_schools.append({"longitude": longitude, "latitude": latitude, "label":0, "closest": -1})

In [ ]:
filepath = './datasets/kmlPublicSchools.kml'
with open(filepath) as fobj:
    folder = parser.parse(fobj).getroot().Document.Folder

public_schools = []
for pm in folder.Placemark:
    coordinates = pm.Point.coordinates
    longitude = float(str(coordinates)[0:11])
    latitude = float(str(coordinates)[12:21])
    if ((latitude > chi_lat_s and latitude < chi_lat_n) and
            (longitude > chi_long_w and longitude < chi_long_e)):
        public_schools.append({"longitude": longitude, "latitude": latitude, "label":0, "closest": -1})

In [ ]:
all_schools = public_schools + private_schools

In [ ]:
for i, school in enumerate(all_schools):
    closest_distance = 1000000
    for index, row in tracts.iterrows():
        dist = get_distance((row['long'], row['lat']),
                            (school["longitude"], school["latitude"]))
        if dist < closest_distance:
            closest_distance = dist
            label = row['label']
            closest_i = index
    all_schools[i]["label"] = int(label)
    all_schools[i]["closest"] = int(closest_i)

In [ ]:
tract_lats = tracts['lat'].tolist()
tract_longs = tracts['long'].tolist()
tract_labels = tracts['label'].tolist()

In [ ]:
tract_dicts = []
for i, vals in enumerate(list(zip(tract_longs, tract_lats, tract_labels))):
    t_dict = {"longitude": vals[0], "latitude": vals[1], "label":vals[2], "closest": i}
    tract_dicts.append(t_dict)

In [ ]:
all_nodes = tract_dicts + hospitals + cta + all_schools

In [ ]:
tract_indices = list(range(0, len(tracts)))
all_indices = list(range(0, len(all_nodes)))
hospital_indices = list(range(len(tracts), len(tracts) + len(hospitals)))
cta_indices = list(range(len(tracts) + len(hospitals), len(tracts) + len(hospitals) + len(cta)))
school_indices = list(range(len(tracts) + len(hospitals) + len(cta), len(tracts) + len(hospitals) + len(cta) + len(all_schools)))

In [ ]:
len(hospitals), len(hospital_indices)

(56, 56)

**Creating the graph using NetworkX**

In [ ]:
city_graph = nx.Graph()
color_map = []
# Create nodes
for i, node_dict in enumerate(all_nodes):
    city_graph.add_node(i,
                        pos=(node_dict["longitude"],
                             node_dict["latitude"])
                        )
    
    if node_dict["label"] == 1:
         color_map.append('purple')
    else:
        color_map.append('lightblue')

# Add hospital edges
for tract_i in all_indices:
    hospital_dict = {}
    for hospital_i in hospital_indices:
        hospital_dict[hospital_i] = get_distance((all_nodes[hospital_i]["longitude"], all_nodes[hospital_i]["latitude"]),
                                             (all_nodes[tract_i]["longitude"], all_nodes[tract_i]["latitude"]))
    
    closest2 = sorted(hospital_dict, key=hospital_dict.get, reverse=False)[0:2]
    
    for i in closest2:
        s_lat = all_nodes[i]["latitude"]
        s_long = all_nodes[i]["longitude"]
        distance = get_distance((all_nodes[tract_i]["longitude"], all_nodes[tract_i]["latitude"]),
                                (s_long, s_lat))
        city_graph.add_edge(tract_i, i, weight=distance)
        # city_graph.add_edge(tract_i, i)

# Add school edges
for tract_i in all_indices:
    school_dict = {}
    for school_i in school_indices:
        school_dict[school_i] = get_distance((all_nodes[school_i]["longitude"], all_nodes[school_i]["latitude"]),
                                             (all_nodes[tract_i]["longitude"], all_nodes[tract_i]["latitude"]))
    
    closest2 = sorted(school_dict, key=school_dict.get, reverse=False)[0:2]
    
    for i in closest2:
        s_lat = all_nodes[i]["latitude"]
        s_long = all_nodes[i]["longitude"]
        distance = get_distance((all_nodes[tract_i]["longitude"], all_nodes[tract_i]["latitude"]),
                                (s_long, s_lat))
        city_graph.add_edge(tract_i, i, weight=distance)
        # city_graph.add_edge(tract_i, i)

# Add subway edges
for tract_i in all_indices:
    cta_dict = {}
    for cta_i in cta_indices:
        cta_dict[cta_i] = get_distance((all_nodes[cta_i]["longitude"], all_nodes[cta_i]["latitude"]),
                                             (all_nodes[tract_i]["longitude"], all_nodes[tract_i]["latitude"]))
    
    closest2 = sorted(cta_dict, key=cta_dict.get, reverse=False)[0:2]
    
    for i in closest2:
        s_lat = all_nodes[i]["latitude"]
        s_long = all_nodes[i]["longitude"]
        distance = get_distance((all_nodes[tract_i]["longitude"], all_nodes[tract_i]["latitude"]),
                                (s_long, s_lat))
        city_graph.add_edge(tract_i, i, weight=distance)
        # city_graph.add_edge(tract_i, i)

In [ ]:
# Create a dictionary of node attributes
attrs = {}
for i, node_dict in enumerate(all_nodes):
    if i in school_indices:
        node_type = [1, 0, 0, 0]
    elif i in hospital_indices:
        note_type = [0, 1, 0, 0]
    elif i in cta_indices:
        node_type = [0, 0, 1, 0]
    else:
        node_type = [0, 0, 0, 1]

    closest_id = node_dict["closest"]

    attrs[i] = {"econ_0": econ_features.iloc[closest_id, 0],
                "econ_1": econ_features.iloc[closest_id, 1],
                "econ_2": econ_features.iloc[closest_id, 2],
                "econ_3": econ_features.iloc[closest_id, 3],
                "econ_4": econ_features.iloc[closest_id, 4],
                "econ_5": econ_features.iloc[closest_id, 5],
                "econ_6": econ_features.iloc[closest_id, 6],
                "econ_7": econ_features.iloc[closest_id, 7],
                "econ_8": econ_features.iloc[closest_id, 8],
                "econ_9": econ_features.iloc[closest_id, 9],
                "econ_10": econ_features.iloc[closest_id, 10],
                "econ_11": econ_features.iloc[closest_id, 11],
                "econ_12": econ_features.iloc[closest_id, 12],
                "econ_13": econ_features.iloc[closest_id, 13],
                "econ_14": econ_features.iloc[closest_id, 14],
                "econ_15": econ_features.iloc[closest_id, 15],
                "econ_16": econ_features.iloc[closest_id, 16],
                "econ_17": econ_features.iloc[closest_id, 17],
                "econ_18": econ_features.iloc[closest_id, 18],
                "econ_19": econ_features.iloc[closest_id, 19],
                "econ_20": econ_features.iloc[closest_id, 20],
                "econ_21": econ_features.iloc[closest_id, 21],
                "econ_22": econ_features.iloc[closest_id, 22],
                "econ_23": econ_features.iloc[closest_id, 23],
                "econ_24": econ_features.iloc[closest_id, 24],
                "econ_25": econ_features.iloc[closest_id, 25],
                "econ_26": econ_features.iloc[closest_id, 26],
                "econ_27": econ_features.iloc[closest_id, 27],
                "econ_28": econ_features.iloc[closest_id, 28],
                "image_0": image_features["shape_area"][closest_id],
                "image_1": image_features['house_num'][closest_id],
                "path_0": path_features.iloc[closest_id, 0],
                "path_1": path_features.iloc[closest_id, 1],
                "path_2": path_features.iloc[closest_id, 2],
                "path_3": path_features.iloc[closest_id, 3],
                "path_4": path_features.iloc[closest_id, 4],
                "path_5": path_features.iloc[closest_id, 5],              
                "node_type_s": node_type[0],
                "node_type_h": node_type[1],
                "node_type_c": node_type[2],
                "node_type_t": node_type[3]
                }


nx.set_node_attributes(city_graph, attrs)

In [ ]:
pip install stellargraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 435 kB 8.0 MB/s 


In [ ]:
import stellargraph as sg
from stellargraph.mapper import GraphSAGENodeGenerator
from stellargraph.layer import GraphSAGE

from tensorflow.keras import layers, optimizers, losses, metrics, Model
import tensorflow as tf

from sklearn import preprocessing, feature_extraction, model_selection
from sklearn.calibration import calibration_curve
from sklearn.linear_model import LogisticRegressionCV
from sklearn.isotonic import IsotonicRegression

from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

from stellargraph.calibration import TemperatureCalibration, IsotonicCalibration
from stellargraph.calibration import plot_reliability_diagram, expected_calibration_error

from stellargraph import datasets
from stellargraph import StellarGraph

In [ ]:
attrs_df = pd.DataFrame(attrs).T
attrs_df

,econ_0,econ_1,econ_2,econ_3,econ_4,econ_5,econ_6,econ_7,econ_8,econ_9,...,path_0,path_1,path_2,path_3,path_4,path_5,node_type_s,node_type_h,node_type_c,node_type_t
0,0.62,33.7,10.3,865.0,495.0,733.0,650.0,1003.0,398.0,4444.0,...,5.0,11.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
1,0.43,34.8,5.3,989.0,934.0,2663.0,714.0,1076.0,644.0,7197.0,...,2.0,10.0,2.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0
2,0.43,36.7,7.0,943.0,185.0,963.0,813.0,1003.0,210.0,2487.0,...,3.0,11.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
3,0.60,42.1,5.5,948.0,372.0,1566.0,597.0,990.0,395.0,6413.0,...,5.0,10.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0
4,0.81,23.8,11.5,979.0,348.0,585.0,651.0,990.0,306.0,5411.0,...,7.0,8.0,1.0,2.0,1.0,2.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2276,0.58,35.6,0.0,1225.0,314.0,802.0,554.0,1355.0,118.0,1727.0,...,10.0,11.0,3.0,4.0,1.0,1.0,1.0,0.0,0.0,0.0
2277,0.48,35.8,5.3,876.0,640.0,61.0,459.0,1010.0,379.0,4805.0,...,7.0,7.0,3.0,4.0,1.0,1.0,1.0,0.0,0.0,0.0
2278,0.46,41.3,4.8,947.0,239.0,38.0,536.0,1024.0,93.0,1780.0,...,8.0,8.0,8.0,10.0,1.0,1.0,1.0,0.0,0.0,0.0
2279,0.35,34.8,19.1,706.0,111.0,10.0,558.0,987.0,111.0,1390.0,...,1.0,5.0,1.0,3.0,1.0,1.0,1.0,0.0,0.0,0.0


In [ ]:
scaler = StandardScaler()

In [ ]:
# Feed the graph into GraphSAGE
sg_city_graph = StellarGraph.from_networkx(city_graph,
                                           node_features=pd.DataFrame(scaler.fit_transform(attrs_df)))

In [ ]:
print(sg_city_graph.info())

StellarGraph: Undirected multigraph
 Nodes: 2281, Edges: 12781

 Node types:
  default: [2281]
    Features: float32 vector, length 41
    Edge types: default-default->default

 Edge types:
    default-default->default: [12781]
        Weights: range=[0, 0.157783], mean=0.0184863, std=0.0204236
        Features: none


In [ ]:
labels = [int(node["label"]) for node in all_nodes]
y = np.zeros([len(labels), 2])

for i, val in enumerate(labels):
    if val == 0:
        y[i][0] = 1
        y[i][1] = 0
    elif val == 1:
        y[i][0] = 0
        y[i][1] = 1
    else:
        print("ERROR")

In [ ]:
batch_size = 150
dim_shape = [16, 4, 2] # Try 32 instead of 16
epochs = 400

In [ ]:
for iteration in range(0, 10):
    train_set, test_set = model_selection.train_test_split(
        pd.DataFrame(y), train_size=0.8, test_size=None, stratify=pd.DataFrame(y)
    )
    val_set = test_set
    target_encoding = preprocessing.LabelBinarizer()
    train_targets = target_encoding.fit_transform(train_set)
    val_targets = target_encoding.fit_transform(val_set)
    test_targets = target_encoding.transform(test_set)

    generator = GraphSAGENodeGenerator(sg_city_graph, batch_size, dim_shape)
    train_gen = generator.flow(train_set.index, train_targets)

    graphsage_model = GraphSAGE(
        layer_sizes=dim_shape, generator=generator, bias=True
    )

    x_inp, x_out = graphsage_model.in_out_tensors()
    prediction = layers.Dense(units=train_targets.shape[1], activation="sigmoid")(x_out)

    model = Model(inputs=x_inp, outputs=prediction)
    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.01),
        loss=losses.binary_crossentropy,
        metrics=["acc"],
    )

    val_gen = generator.flow(val_set.index, val_targets)
    test_gen = generator.flow(test_set.index, test_targets)

    history = model.fit(
        train_gen, epochs=epochs, validation_data=val_gen, shuffle=True,
    )

    all_gen = generator.flow(list(city_graph.nodes), y)
    all_predictions = model.predict(all_gen).squeeze()[..., np.newaxis]
    all_predictions_df = pd.DataFrame(all_predictions[:,0], index=list(list(city_graph.nodes)))
    test_preds = all_predictions_df.loc[test_set.index, :]

    test_predictions = test_preds.values
    test_predictions_class = ((test_predictions > 0.5) * 1).flatten()
    test_df = pd.DataFrame(
        {
            "Predicted_score": test_predictions.flatten(),
            "Predicted_class": test_predictions_class,
            "True": test_targets[:, 0],
        }
    )
    
    if iteration == 0:
        all_vals_df = pd.DataFrame(
            {
                "Predicted_score": test_predictions.flatten(),
                "Predicted_class": test_predictions_class,
                "True": test_targets[:, 0],
            }
        )
    else:
        all_vals_df = pd.concat([all_vals_df, test_df], ignore_index=True, sort=False)

    print(iteration)

Streaming output truncated to the last 5000 lines.
13/13 [==============================] - 4s 348ms/step - loss: 0.1573 - acc: 0.9474 - val_loss: 0.2865 - val_acc: 0.9015
Epoch 305/400
13/13 [==============================] - 4s 345ms/step - loss: 0.1592 - acc: 0.9441 - val_loss: 0.3459 - val_acc: 0.8818
Epoch 306/400
13/13 [==============================] - 5s 350ms/step - loss: 0.1775 - acc: 0.9375 - val_loss: 0.3099 - val_acc: 0.8972
Epoch 307/400
13/13 [==============================] - 4s 368ms/step - loss: 0.1659 - acc: 0.9474 - val_loss: 0.2890 - val_acc: 0.8906
Epoch 308/400
13/13 [==============================] - 4s 347ms/step - loss: 0.1564 - acc: 0.9479 - val_loss: 0.2972 - val_acc: 0.8928
Epoch 309/400
13/13 [==============================] - 4s 346ms/step - loss: 0.1511 - acc: 0.9496 - val_loss: 0.2914 - val_acc: 0.8993
Epoch 310/400
13/13 [==============================] - 5s 352ms/step - loss: 0.1495 - acc: 0.9452 - val_loss: 0.3071 - val_acc: 0.8928
Epoch 311/400
13/1

In [ ]:
all_vals_df

,Predicted_score,Predicted_class,True
0,0.982978,1,1
1,0.982564,1,0
2,0.982978,1,1
3,0.985220,1,1
4,0.477151,0,1
...,...,...,...
4565,0.998147,1,1
4566,0.993132,1,1
4567,0.996420,1,1
4568,0.999148,1,1


In [ ]:
all_vals_df.to_csv("./datasets/chicago_graphsage.csv", encoding='utf-8', index=False)